<div align="left"><img width="15%" src="Materials/Logo.png"></div>

# __Python and Datascience Workshop__
Author: Mohammad Akradi <sup>1</sup> <br>
<sup>1</sup> Institute of Medical Science and Technology, Shahid Beheshti University, Tehran, Iran

## __Session 2a__
The primary goal of this section is to become familiar with loading, modifying, saving, and visualizing neuroimages in Python. A secondary goal is to develop a conceptual understanding of the data structures involved, to facilitate diagnosing problems in data or analysis pipelines.

## __Nibabel__ <img style="float: right;" src="Materials/nibabel-logo.svg">
[Reference](https://nipy.org/nibabel/) <br>

This package provides read +/- write access to some common medical and neuroimaging file formats, including: ANALYZE (plain, SPM99, SPM2 and later), GIFTI, NIfTI1, NIfTI2, CIFTI-2, MINC1, MINC2, AFNI BRIK/HEAD, MGH and ECAT as well as Philips PAR/REC. We can read and write FreeSurfer geometry, annotation and morphometry files. There is some very limited support for DICOM. NiBabel is the successor of PyNIfTI.

in this workshop we will focus on `nifti` file formats. however, almost all of these formats can be loaded by `nibabel.load` function.

* Installation

```python
!pip install nibabel nilearn
```

* Setup

```python
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nb
```

* Loading data

```python
# Load
img = nb.load('data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz')
# Loog at the header
print(img)
```
If you're interested in specific parameters, you can access them very easily, as the following examples show.

* Access specific parameeters

```python
# shape of data
data = img.get_fdata()
print(data.shape)

# affine martix
affine = img.affine
print(affine)

# header
header = img.header['pixdim']
print(header)
```

when you run above commands, Note that in the 'pixdim' above contains the voxel resolution (4., 4., 3.999), as well as the TR (2.5).

The data is a simple numpy array. It has a shape, it can be sliced and generally manipulated as you would any array.

The affine is a 4 x 4 numpy array. This describes the transformation from the voxel space (indices [i, j, k]) to the reference space (distance in mm (x, y, z)).

It can be used, for instance, to discover the voxel that contains the origin of the image.

* Affine

```python
x, y, z, _ = np.linalg.pinv(affine).dot(np.array([0, 0, 0, 1])).astype(int)

print("Affine:\n", affine)
print(f"\nCenter: {x}, {y}, {z}")
```

The affine also encodes the axis orientation and voxel sizes:

```python
print(nb.aff2axcodes(affine))
print(nb.affines.voxel_sizes(affine))
print()
```

* Visualizing data with matplotlib

```python
data = img.get_fdata()
plt.imshow(data[:, :, 15, 0].T, cmap='Greys_r')
print(data.shape)
```

now we will load anatomical image of data and visualize it:

```python
t1 = nb.load('data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz')
data = t1.get_fdata()
plt.imshow(data[:, :, data.shape[2] // 2].T, cmap='Greys_r')
print(data.shape)
```

Nibabel has its own viewer, which can be accessed through `img.orthoview()`. This viewer scales voxels to reflect their size, and labels orientations.

* Orthoview

```python
t1.orthoview()
```

* `nib-ls`

Nibabel comes packaged with a command-line tool to print common metadata about any (volumetric) neuroimaging format nibabel supports. By default, it shows (on-disk) data type, dimensions and voxel sizes.

```python
!nib-ls data/002_S_0295/Session_0/*

!nib-ls -H descrip data/ds000114/sub-01/ses-test/*/*.nii.gz
```

If we processed our image (which we will learn in the next section), then we will need to save the processed image as a nifti file. to this approach, we will use `nb.save` function.

* Processing data with numpy and Save as nifti

```python
# First, we need to load the image and get the data
img = nb.load('data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz')
data = img.get_fdata()

# Now we force the values to be between 0 and 255
# and change the datatype to unsigned 8-bit
rescaled = ((data - data.min()) * 255. / (data.max() - data.min())).astype(np.uint8)

# Now we can save the changed data into a new NIfTI file
new_img = nb.Nifti1Image(rescaled, affine=img.affine, header=img.header)
nb.save(new_img, 'data/rescaled_image.nii.gz')
```

## __Nilearn__ <img style="float: right;" src="Materials/nilearn-logo2.png">
[Reference](https://nilearn.github.io/stable/index.html) <br>

Nilearn enables approachable and versatile analyses of brain volumes. It provides statistical and machine-learning tools, with instructive documentation & open community. <br>
It supports general linear model (GLM) based analysis and leverages the scikit-learn Python toolbox for multivariate statistics with applications such as predictive modelling, classification, decoding, or connectivity analysis.

* Installation

```python
!pip install nilearn
```

* Setup

```python
import nilearn as nl
from nilearn import plotting
from nilearn import image
import nibabel as nb
import pylab as plt
import numpy as np
```

In [4]:
import nibabel as nb
import matplotlib.pyplot as plt
t1 = nb.load('data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz')

In [11]:
!pip uninstall dask-labextension

Found existing installation: dask-labextension 5.2.0
Uninstalling dask-labextension-5.2.0:
  Would remove:
    /home/mohammad/DataScience/Python/etc/jupyter/jupyter_notebook_config.d/dask_labextension.json
    /home/mohammad/DataScience/Python/etc/jupyter/jupyter_server_config.d/dask_labextension.json
    /home/mohammad/DataScience/Python/lib/python3.8/site-packages/dask_labextension-5.2.0.dist-info/*
    /home/mohammad/DataScience/Python/lib/python3.8/site-packages/dask_labextension/*
    /home/mohammad/DataScience/Python/share/jupyter/labextensions/dask-labextension/install.json
    /home/mohammad/DataScience/Python/share/jupyter/labextensions/dask-labextension/package.json
    /home/mohammad/DataScience/Python/share/jupyter/labextensions/dask-labextension/schemas/dask-labextension/package.json.orig
    /home/mohammad/DataScience/Python/share/jupyter/labextensions/dask-labextension/schemas/dask-labextension/plugin.json
    /home/mohammad/DataScience/Python/share/jupyter/labextensions

In [12]:
Y


NameError: name 'Y' is not defined